In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import datasets
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [17]:
df = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/clean_data.csv', delimiter=',')


In [9]:
df

,evap_cfs,inflow_cfs,elevation_feet,ppt_feet,storage_ft,outflow_cfs,inflow_catego,outflow_catego
0,1.230449,3.382737,2.866925,0.000000,13.829758,3.550840,drought,okay
1,1.113943,3.318481,2.866701,0.000000,13.828818,3.585122,drought,okay
2,1.591065,3.204663,2.866465,0.000000,13.827829,3.523616,drought,okay
3,1.361728,3.304706,2.866234,0.000000,13.826863,3.570776,drought,okay
4,1.361728,3.281488,2.865986,0.000000,13.825822,3.568554,drought,okay
...,...,...,...,...,...,...,...,...
1798,1.176091,3.480438,2.820851,-2.380211,13.633561,3.301030,okay,drought
1799,1.278754,3.546296,2.821160,-1.026103,13.634895,3.301030,okay,drought
1800,1.342423,3.566437,2.821507,0.000000,13.636399,3.278754,okay,drought
1801,1.000000,3.413970,2.821664,-2.234083,13.637079,3.255273,drought,drought


## Do outflow first

In [18]:
df=df.drop(['inflow_catego','ppt_feet','outflow_cfs'], axis=1)
df

,evap_cfs,inflow_cfs,elevation_feet,storage_ft,outflow_catego
0,1.230449,3.382737,2.866925,13.829758,okay
1,1.113943,3.318481,2.866701,13.828818,okay
2,1.591065,3.204663,2.866465,13.827829,okay
3,1.361728,3.304706,2.866234,13.826863,okay
4,1.361728,3.281488,2.865986,13.825822,okay
...,...,...,...,...,...
1798,1.176091,3.480438,2.820851,13.633561,drought
1799,1.278754,3.546296,2.821160,13.634895,drought
1800,1.342423,3.566437,2.821507,13.636399,drought
1801,1.000000,3.413970,2.821664,13.637079,drought


In [19]:
np.random.seed(1)
tf.random.set_seed(1)

train, test = train_test_split(df, train_size=0.7, random_state=1)

def create_model(layer1=3, layer2=3):
    opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.0, nesterov=False, name="SGD")
    
    model = keras.Sequential()
    model.add(Dense(layer1, input_shape=(4,)))
    model.add(Activation('relu')) 
    model.add(Dense(layer2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'],) 
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model)
param_grid = {'layer1':[3, 5], 'layer2':[3, 5]}
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(train.drop(columns=['outflow_catego']), train['outflow_catego'])
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

40/40 [==============================] - 0s 799us/step - loss: nan - accuracy: 0.1289
Best: 0.317743 using {'layer1': 3, 'layer2': 3}


In [104]:
model1 = create_model(layer1 = 3, layer2 = 3)

In [105]:
from numpy import array

Xnew = array([[2, 5, 3,6.5]])
# make a prediction
ynew = np.argmax(model1.predict(Xnew), axis=-1)
# show the inputs and predicted outputs
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

X=[2.  5.  3.  6.5], Predicted=0


In [108]:
train

,evap_cfs,inflow_cfs,elevation_feet,storage_taf,outflow_categ
1436,1.968483,2.820858,2.862400,6.171681,ok
1147,0.845098,3.571243,2.899941,6.327812,ok
1180,1.591065,3.353147,2.906028,6.352878,ok
924,2.130334,3.944828,2.951090,6.536021,ok
547,2.060698,3.559907,2.915985,6.393701,ok
...,...,...,...,...,...
905,1.929419,4.090011,2.949873,6.531122,ok
1791,1.146128,3.714497,2.818681,5.985067,ok
1096,0.903090,3.308137,2.889391,6.284209,ok
235,2.075547,3.529943,2.909577,6.367459,ok


In [55]:
X = df.iloc[160:170]
y = df['outflow_categ'].iloc[160:170]
y

160    medium
161    medium
162    medium
163    medium
164    medium
165    medium
166    medium
167    medium
168    medium
169    medium
Name: outflow_categ, dtype: object

In [57]:
X[2]

KeyError: 2